# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?
2. 리뷰어는 어디서 가져오지?
3. 연도를 어떻게 하지 ? 연도가 없는 경우도 있고, 연도와 요일이 없는 경우도 있음 

### 개요 

1. 매장 정보 크롤링 결과에서 업태구분 별로 리뷰가 많은 식당 n개를 골라 가져온다
2. 리뷰가 많은 식당 n개에서 각각 3명씩 ( 추천순 or 최신순 일지는 아직 미정) 

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 1. 그룹 데이터에서 검색어 사용( 정렬이 완료되었다는 가정 하에)

In [2]:
res_group_df = pd.read_excel('매장_groupby_test.xlsx')
res_group_df


,업태구분,Unnamed: 1,식당이름,업태구분.1,주소,메뉴,가격,리뷰,검색어
0,PC방,19,제로백PC 매직엔 개포점,PC방,서울 강남구 개포로 223 지층,메뉴없음,가격없음,86,"개포로 223, 매직엔 PC방"
1,"곱창,막창,양",22,만득이곱창,"곱창,막창,양",서울 강남구 강남대로124길 36 1층 만득이곱창,야채곱창(1~2인분),"16,000원",321,"강남대로124길 36, 만득이곱창"
2,NaN,3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",315,"학동로101길 7, 삼성양곱창"
3,김밥,2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥
4,돈가스,14,혼돈,돈가스,서울 강남구 언주로115길 15 101호,멘치카츠,"13,000원",114,"언주로115길 15, 혼돈"
5,돼지고기구이,9,길목 신관,돼지고기구이,서울 강남구 봉은사로 504 진양빌딩 1층,투뿔목살,"18,000원",54,"봉은사로 504, 길목 신관"
6,"백반,가정식",5,다미가정식백반,"백반,가정식",서울 강남구 개포로20길 16,가정식백반,"9,000원",53,개포로20길 16 다미
7,뷔페,12,더 키푸드,뷔페,서울 강남구 선릉로 823 6층,정찬,"10,000원",87,"선릉로 823, 더키푸드"
8,"순대,순댓국",18,계경순대국 수서역벤처빌점,"순대,순댓국",서울 강남구 밤고개로1길 10 수서현대벤쳐빌 지하 2층 202호,순대국,"8,000원",371,"밤고개로1길 10, 계경순대국수서현대벤쳐빌점"
9,양식,1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",302,테헤란로 124 뢰벤 돈까스


### 전처리 필요 없애기 ( vscode 컨디션 안좋아서 크롤링 안된경우 있음)



In [3]:
res_group_df = res_group_df.loc[res_group_df['주소']!='전처리필요',:]
res_group_df

,업태구분,Unnamed: 1,식당이름,업태구분.1,주소,메뉴,가격,리뷰,검색어
0,PC방,19,제로백PC 매직엔 개포점,PC방,서울 강남구 개포로 223 지층,메뉴없음,가격없음,86,"개포로 223, 매직엔 PC방"
1,"곱창,막창,양",22,만득이곱창,"곱창,막창,양",서울 강남구 강남대로124길 36 1층 만득이곱창,야채곱창(1~2인분),"16,000원",321,"강남대로124길 36, 만득이곱창"
2,NaN,3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",315,"학동로101길 7, 삼성양곱창"
3,김밥,2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥
4,돈가스,14,혼돈,돈가스,서울 강남구 언주로115길 15 101호,멘치카츠,"13,000원",114,"언주로115길 15, 혼돈"
5,돼지고기구이,9,길목 신관,돼지고기구이,서울 강남구 봉은사로 504 진양빌딩 1층,투뿔목살,"18,000원",54,"봉은사로 504, 길목 신관"
6,"백반,가정식",5,다미가정식백반,"백반,가정식",서울 강남구 개포로20길 16,가정식백반,"9,000원",53,개포로20길 16 다미
7,뷔페,12,더 키푸드,뷔페,서울 강남구 선릉로 823 6층,정찬,"10,000원",87,"선릉로 823, 더키푸드"
8,"순대,순댓국",18,계경순대국 수서역벤처빌점,"순대,순댓국",서울 강남구 밤고개로1길 10 수서현대벤쳐빌 지하 2층 202호,순대국,"8,000원",371,"밤고개로1길 10, 계경순대국수서현대벤쳐빌점"
9,양식,1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",302,테헤란로 124 뢰벤 돈까스


In [7]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [14]:
from selenium.webdriver.common.action_chains import ActionChains

In [15]:
name = '개포로 223, 매직엔 PC방'
driver.get('https://map.naver.com/p/search/{}'.format(name))
time.sleep(3)  
try :
    if driver.find_elements(By.ID,'entryIframe') :
        entryIframe = driver.find_element(By.ID,'entryIframe')
        driver.switch_to.frame(entryIframe)
except :
    pass 

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for _ in range(4):
# 스크롤을 위한 JavaScript 코드 실행
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 
num_click = 3

for _ in range(num_click):
    try:
        # 태그 및 리뷰 클래스 찾기
        tag_element = driver.find_element(By.CLASS_NAME, 'ExHfk.Ay7tv')
        
        # 태그가 보이는 위치로 스크롤
        actions = ActionChains(driver)
        actions.move_to_element(tag_element).perform()
        
        # 클릭하기
        tag_element.click()
        time.sleep(1)
        
        review_element = driver.find_element(By.CLASS_NAME, 'Ky28p')
        
        # 리뷰가 보이는 위치로 스크롤
        actions.move_to_element(review_element).perform()
        
        # 클릭하기
        review_element.click()
        time.sleep(1)
    except Exception as e:
        print("An error occurred:", e)



In [ ]:
# ### 일단 myplace에 접속하는 거 필요 

# for i in range(0,len(res_group_df)):
#     # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
#     name = res_group_df['검색어'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))



User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

In [4]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 
def user_profile(url) :
    # 주소 이동
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['주소'] = url

    print(user_element)

    return user_element


일자, 요일, 방문일수 분류 함수 

In [5]:
def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')
    date = date_part[0]
    weekday = date[-3:]
    revisit = int(date_part[1].replace('번째 방문', ''))

    # 일자 타입 변경하기 
    date = '24' + date if date.startswith('년') else date # 만약 연도가 없는 경우 24를 붙이기
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    year, month, day = map(int, match[0])

    # 날짜 객체로 변환
    date_object = datetime(2000+year, month, day) 
    formatted_date = date_object.strftime('%Y-%m-%d')

    return formatted_date, weekday, revisit



### 유저 정보 찾기

In [6]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()

# User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
user_data = user_profile(url)

# User 정보 저장 
user_profile_df = pd.DataFrame([user_data], index=[0])
user_profile_df

{'아이디': 'xll****', '리뷰': 983, '팔로워': 75, '팔로잉': 0, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}


,아이디,리뷰,팔로워,팔로잉,주소
0,xll****,983,75,0,https://m.place.naver.com/my/5c36b9f1e511a8856...


### 매장 정보 찾기

In [7]:
# 맨 처음 게시물 클릭 
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
# data를 담을 빈 리스트 선언
data_list = [] 

# 페이지 아래로 스크롤 몇번 
page_down = 5

# 페이지 스크롤 횟수만큼 반복
for _ in range(page_down):

    # 대기시간 5초 
    wait = WebDriverWait(driver, 5)

    # 요소를 찾을 때 대기 조건 추가
    user_review_elements = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
    )

    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

    # 리뷰에서 정보 가져오기
    for reviews_elements in user_review_elements:
        i = 0 # 카테고리와 주소 구분할 때 사용
        restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
        
        if bool(restaurant_elements): # 정보가 없는 경우가 있음

            # 매장명 찾기
            restaurant_name = restaurant_elements[0].text  

    #<--------------------------------------------------------------------------------------------------->
            # 카테고리와 주소 찾기 
            category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
            category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
            
            # span 태그 안에 있는 텍스트 가져오기
            span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
            category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
            location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
            i += 1
    #<--------------------------------------------------------------------------------------------------->
            # 리뷰 찾기 
            review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
            try : 
                review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문          
                sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                if bool(sub_info):
                    sub_info=sub_info[0].text
                else:
                    sub_info=None
            except :
                review_text = '리뷰 없음'
                sub_info = ''

    #<--------------------------------------------------------------------------------------------------->
            # 태그 찾기 
            # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
            reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
            command=False
            for x in reactions_elements:
                if x.get_attribute("role"): # 리액션 숨김 없애기
                    x.click()
                    command=True
                else:
                    continue
            if command:
                reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
            reactions=list(map(lambda x:x.text,reactions_elements))
            # 리스트를 벗김 
            reactions = str(reactions).replace('[', '').replace(']', '')

    #<--------------------------------------------------------------------------------------------------->
            # 방문 일자, 재방문 이력 찾기
            date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
            date=[x.text for x in date_info]
            day,weekday,revisit = change_date_format(date) # 함수 사용
        
            # 정보 추가
            data_dict = {
                    '아이디' : user_data['아이디'],
                    '리뷰' : user_data['리뷰'],
                    '팔로워' : user_data['팔로워'],
                    '매장명': restaurant_name,
                    '카테고리': category,
                    '주소' : location,
                    '리뷰 내용': review_text,
                    "세부정보" : sub_info,
                    '태그': reactions,
                    '방문일자': day,
                    '요일' : weekday,
                    '재방문횟수':revisit}
            data_list.append(data_dict)
    user_df = pd.DataFrame(data_list)
    user_df = user_df.drop_duplicates()
  
user_df.head(5)

KeyboardInterrupt: 

유저 리뷰 가져오는 함수

In [10]:
def find_user_data(page_down,driver) : 
    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # data를 담을 빈 리스트 선언
    data_list = [] 


    # 페이지 스크롤 횟수만큼 반복
    for _ in range(page_down):

        # 대기시간 5초 
        wait = WebDriverWait(driver, 5)

        # 요소를 찾을 때 대기 조건 추가
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
        )

        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
            
            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                
                # span 태그 안에 있는 텍스트 가져오기
                span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                i += 1
        #<--------------------------------------------------------------------------------------------------->
                # 리뷰 찾기 
                review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                try : 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문          
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None
                except :
                    review_text = '리뷰 없음'
                    sub_info = ''

        #<--------------------------------------------------------------------------------------------------->
                # 태그 찾기 
                # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
                reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
                command=False
                for x in reactions_elements:
                    if x.get_attribute("role"): # 리액션 숨김 없애기
                        x.click()
                        command=True
                    else:
                        continue
                if command:
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                reactions=list(map(lambda x:x.text,reactions_elements))
                # 리스트를 벗김 
                reactions = str(reactions).replace('[', '').replace(']', '')

        #<--------------------------------------------------------------------------------------------------->
                # 방문 일자, 재방문 이력 찾기
                date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                date=[x.text for x in date_info]
                day,weekday,revisit = change_date_format(date) # 함수 사용
            
                # 정보 추가
                data_dict = {
                        '아이디' : user_data['아이디'],
                        '리뷰' : user_data['리뷰'],
                        '팔로워' : user_data['팔로워'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit}
                data_list.append(data_dict)
        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()
  
    return user_df



# 위 내용을 반복문으로 한번에 하기 

In [11]:
# 네이버에서 찾아온 리뷰어들의 주소 
# myplace url
user_list = ['https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2',
'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2',
'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2',
'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2',
'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2',
'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/review?v=2',
'https://m.place.naver.com/my/5cf4c309c1dd7fdcdfcd76ac/review?v=2',
'https://m.place.naver.com/my/5e7eca2e8f87a842bcb991f4/review?v=2',
'https://m.place.naver.com/my/5efde03915c4bcd430585d15/review?v=2',
'https://m.place.naver.com/my/5e7ad86b8f87a842bc214c88/review?v=2',
'https://m.place.naver.com/my/5dda92cd9ec8258e4a9d1fe8/review?v=2',
'https://m.place.naver.com/my/5e0b47d58f87a842bce9ecb3/review?v=2',
'https://m.place.naver.com/my/614261c038bd206d952895c4/review?v=2',
'https://m.place.naver.com/my/5b7e88d07462e986799ec864/review?v=2',
'https://m.place.naver.com/my/5d251d01dd1a688aca746736/review?v=2',
'https://m.place.naver.com/my/641bd1d71438fc3a9c38c380/review?v=2',
'https://m.place.naver.com/my/5c05f3f72a1c47487f7761cd/review?v=2',
'https://m.place.naver.com/my/5ca9640d35617da4eb6e4de2/review?v=2',
'https://m.place.naver.com/my/5df5ac8a8f87a842bc8f5a9d/review?v=2',
'https://m.place.naver.com/my/63bfe830b46be2000656013b/review?v=2',
'https://m.place.naver.com/my/5df643fa8f87a842bcd031e1/review?v=2'
]


In [12]:


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) # 에러나면 driver = webdriver.Chrome()
# 스크롤 횟수 
page_down = 3

# 데이터프레임 선언
profile_df = pd.DataFrame()
total_user_df =  pd.DataFrame()

# 반복해서 url 정보를 가져오자~ 
for url in user_list :
    try :     
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수 
        user_data = user_profile(url)
        # user_profile_df = pd.DataFrame([user_data], index=[0])
        if 'user_profile_df' in locals() and not user_profile_df.empty:
            user_profile_df = user_profile_df.append(user_data, ignore_index=True)
        else:
            user_profile_df = pd.DataFrame([user_data], index=[0])
            
        # User review 정보를 가져오는 함수
        user_df = find_user_data(page_down,driver)

        # User review 정보 저장 
        total_user_df = pd.concat([total_user_df, user_df], ignore_index=True)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue

{'아이디': 'xll****', '리뷰': 983, '팔로워': 75, '팔로잉': 0, '주소': 'https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2'}
에러 메시지: Message: stale element reference: stale element not found
  (Session info: chrome=122.0.6261.69)

{'아이디': '맛있는거 먹으려고 운동함', '리뷰': 534, '팔로워': 42, '팔로잉': 2, '주소': 'https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2'}
{'아이디': '참참s', '리뷰': 507, '팔로워': 75, '팔로잉': 0, '주소': 'https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2'}
{'아이디': 'JUDY0725', '리뷰': 1932, '팔로워': 97, '팔로잉': 0, '주소': 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2'}
에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3P-5HQ"}
  (Session info: chrome=122.0.6261.69)

{'아이디': '초코피스타치오', '리뷰': 169, '팔로워': 24, '팔로잉': 6, '주소': 'https://m.place.naver.com/my/6010e880e71246c530be8c27/review?v=2'}
{'아이디': 'sodadada7', '리뷰': 183, '팔로워': 0, '팔로잉': 1, '주소': 'https://m.place.naver.com/my/5e27e4ec8f87a842bcb3505c/revi

In [ ]:
user_profile_df

,아이디,리뷰,팔로워,팔로잉,주소
0,xll****,983,75,0,https://m.place.naver.com/my/5c36b9f1e511a8856...
1,맛있는거 먹으려고 운동함,534,42,2,https://m.place.naver.com/my/5e1370ce8f87a842b...
2,참참s,507,75,0,https://m.place.naver.com/my/5f1dd9049ec8258e4...
3,JUDY0725,1932,97,0,https://m.place.naver.com/my/5bf92274b7236e377...
4,초코피스타치오,169,24,6,https://m.place.naver.com/my/6010e880e71246c53...
5,sodadada7,183,0,1,https://m.place.naver.com/my/5e27e4ec8f87a842b...
6,포도267,667,13,0,https://m.place.naver.com/my/5cf4c309c1dd7fdcd...
7,BBOK365,1805,21,10,https://m.place.naver.com/my/5e7eca2e8f87a842b...
8,맛집인기가요,813,2786,0,https://m.place.naver.com/my/5efde03915c4bcd43...
9,janeyi,777,49,0,https://m.place.naver.com/my/5e7ad86b8f87a842b...


In [18]:
total_user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,맛있는거 먹으려고 운동함,534,42,메디큐브의원 강남신논현역,피부과,서울특별시 서초구 서초동,"강남역 피부과입니다! 신논현역, 강남역에서 도보로 10분 내로 방문할 수 있어서 접...",이용 방법예약 후대기 시간바로 입장,,2024-02-05,월요일,3
1,맛있는거 먹으려고 운동함,534,42,관계,요리주점,서울특별시 강남구 신사동,"예약 후 방문 했습니다.\n김치전, 술국, 계란찜 등 안주는 전부다 맛있었어요! \...",이용 방법예약 후대기 시간바로 입장목적회식동행친구,"'음식이 맛있어요', '특별한 메뉴가 있어요'",2023-12-29,금요일,1
2,맛있는거 먹으려고 운동함,534,42,디라이프스타일키친 광화문점,양식,서울특별시 중구 태평로1가,"- 분위기: 연말 느낌도 나고, 화려하게 꾸며져있어서 좋았습니다\n- 맛: 파스타,...",이용 방법예약 없이대기 시간30분 이내목적데이트동행연인·배우자,'음식이 맛있어요',2023-12-22,금요일,1
3,맛있는거 먹으려고 운동함,534,42,청담이상 강남역점,이자카야,서울특별시 강남구 역삼동,"송년회 2차로 방문했습니다. 분위기 좋고, 안주도 이만하면 만족스러웠습니다 ^^",이용 방법예약 없이대기 시간바로 입장목적회식동행친구,"'인테리어가 멋져요', '단체모임 하기 좋아요'",2023-12-21,목요일,1
4,맛있는거 먹으려고 운동함,534,42,길목,돼지고기구이,서울특별시 강남구 삼성동,역시 길목 목살은 최고입니다! \n덕분에 행복한 생일 식사를 할 수 있었어요 😁,이용 방법예약 없이대기 시간30분 이내목적데이트동행연인·배우자,'음식이 맛있어요',2023-12-02,토요일,2
...,...,...,...,...,...,...,...,...,...,...,...,...
364,노이노,206,15,냠냠족발&보쌈 강남논현본점,"족발,보쌈",서울특별시 강남구 논현동,가성비 대박인 신논현 족발 맛집 !!!! 족발만 시키면 2만원대인데 저희는 3종류시...,이용 방법예약 후대기 시간바로 입장목적데이트동행연인·배우자,"'음식이 맛있어요', '가성비가 좋아요', '인테리어가 멋져요', '매장이 넓어요'...",2024-01-30,화요일,1
365,노이노,206,15,나날스킨,"피부,체형관리",서울특별시 강남구 논현동,논현 피부관리 받으러 와서 처음으로 슈퍼젝션 관리를 받아봤는데.. 정말 눈물나게 아...,None,"'관리 효과가 좋아요', '맞춤 케어를 잘해줘요', '시술이 꼼꼼해요', '친절해요...",2024-01-30,화요일,1
366,노이노,206,15,용용선생 선릉점,요리주점,서울특별시 강남구 대치동,믿고가는 용용선생.. 선릉점은 다른곳과 다르게 2층까지 있더라구요! 단체석 많아서 ...,이용 방법예약 후대기 시간바로 입장목적친목동행친구,"'음식이 맛있어요', '가성비가 좋아요', '인테리어가 멋져요', '대화하기 좋아요...",2024-01-24,수요일,1
367,노이노,206,15,윤이프뷰티,"네일아트,네일샵",서울특별시 강남구 역삼동,시술 너무 이쁘게 잘 받고왔습니다! 속눈썹 결도 넘 이쁘게 잘 나왔고 확실히 케라틴...,None,"'원하는 디자인을 잘해줘요', '유지력이 좋아요', '시술이 꼼꼼해요', '친절해요...",2024-01-22,월요일,1


In [19]:
total_user_df.to_excel('data/user_df_test.xlsx', index=False)

In [15]:
# 유저의 리뷰
total_user_df.to_excel('data/user_df_test.xlsx', index=False)

# # 유저의 정보 
# user_profile_df.to_excel('data/user_profile_df_test.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'data/user_df_test.xlsx'

In [ ]:
# 리셋
data_list = []
del user_profile_df
del total_user_df